# Project 4

## Retrieve data Using PySpark

In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.0'
spark_version = 'spark-3.5.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,321 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,622 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/

In [2]:
# Import packages
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import StructType,StructField,StringType, DateType,IntegerType

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles

path = r'/content/drive/Othercomputers/My PC/Data Analytics Bootcamp/23-Project-4-Week-1/0_Project/'
filepath = path + "Project-4/Resources/winequality-red.csv"

print(filepath)

spark.sparkContext.addFile(filepath)
df = spark.read.csv(SparkFiles.get("winequality-red.csv"), header=True, inferSchema=True, quote="\"", escape="\"")

# Show DataFrame
df.show()

/content/drive/Othercomputers/My PC/Data Analytics Bootcamp/23-Project-4-Week-1/0_Project/Project-4/Resources/winequality-red.csv
+-------------+----------------+-----------+--------------+-------------------+-------------------+--------------------+-------+----+---------+-------+-------+
|fixed acidity|volatile acidity|citric acid|residual sugar|          chlorides|free sulfur dioxide|total sulfur dioxide|density|  pH|sulphates|alcohol|quality|
+-------------+----------------+-----------+--------------+-------------------+-------------------+--------------------+-------+----+---------+-------+-------+
|          7.4|             0.7|        0.0|           1.9|              0.076|               11.0|                34.0| 0.9978|3.51|     0.56|    9.4|      5|
|          7.8|            0.88|        0.0|           2.6|              0.098|               25.0|                67.0| 0.9968| 3.2|     0.68|    9.8|      5|
|          7.8|            0.76|       0.04|           2.3|           

In [5]:
#Create a temporary view.
df.createOrReplaceTempView('wine_data')

In [6]:
import pandas as pd

pandas_df=spark.sql("""
SELECT * from wine_data
""").toPandas()

# pandas_df.to_csv('top5Gold.csv')

In [102]:
pandas_df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,target
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,0
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,0


## Check data and clean if required

In [104]:
pandas_df["quality"].max()

8

In [105]:
pandas_df["quality"].min()

3

In [106]:
pandas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int32  
 12  target                1599 non-null   object 
dtypes: float64(11), int32(1), object(1)
memory usage: 156.3+ KB


In [9]:
pandas_df.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


In [10]:
pandas_df["quality"].value_counts()

5    681
6    638
7    199
4     53
8     18
3     10
Name: quality, dtype: int64

In [ ]:
pandas_df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


### Everything looks good and clean. Let's proceed

## Define Target based on quality rating: 7 or above =  good. Otherwise bad.

In [107]:

target = []
for i in pandas_df['quality']:
    if i >= 7:
        target.append('1')
    else:
        target.append('0')
pandas_df['target'] = target

target_names = ['bad quality', 'good quality']

pandas_df.head()


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,target
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,0
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,0


In [108]:
pandas_df['target'].unique()

array(['0', '1'], dtype=object)

In [109]:
X = pandas_df.iloc[:,:11]
y = pandas_df['target']

In [110]:
X.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


In [111]:
y.head()

0    0
1    0
2    0
3    0
4    0
Name: target, dtype: object

In [113]:
#features
print(X)

      fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0               7.4             0.700         0.00             1.9      0.076   
1               7.8             0.880         0.00             2.6      0.098   
2               7.8             0.760         0.04             2.3      0.092   
3              11.2             0.280         0.56             1.9      0.075   
4               7.4             0.700         0.00             1.9      0.076   
...             ...               ...          ...             ...        ...   
1594            6.2             0.600         0.08             2.0      0.090   
1595            5.9             0.550         0.10             2.2      0.062   
1596            6.3             0.510         0.13             2.3      0.076   
1597            5.9             0.645         0.12             2.0      0.075   
1598            6.0             0.310         0.47             3.6      0.067   

      free sulfur dioxide  

In [114]:
#target
y.value_counts()


0    1382
1     217
Name: target, dtype: int64

## Preprocessing: Split test and training data & Standard Scaler

In [116]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


In [117]:

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

In [118]:
# Fitting Standard Scaler
X_scaler = scaler.fit(X_train)

In [119]:
# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Run Models

## Some imports

In [124]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, balanced_accuracy_score
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import RandomOverSampler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

## Logistic Regression - `solver='lbfgs'`

In [136]:
# instantiate the Logistic Regression model as classifier

classifier = LogisticRegression(solver='lbfgs',
                                max_iter=200,
                                random_state=1)
classifier

LogisticRegression(max_iter=200, random_state=1)

In [137]:
# Fit model using scaled training data

lr_model = classifier.fit(X_train_scaled, y_train)
lr_model


LogisticRegression(max_iter=200, random_state=1)

## Create predictded values for test and train

In [138]:
#Generate training predictions
# training_predictions = lr_model.predict(X_train.values)

#Generate testing predictions
testing_predictions = classifier.predict(X_test.values)

## Confusion Matrix for testing data

In [139]:
# Create and save the confusion matrix for the testing data
test_matrix = confusion_matrix(y_test, testing_predictions)

# Print the confusion matrix for the testing data
print(test_matrix)

[[317  38]
 [ 32  13]]


## Classification report for testing

In [140]:
# Create and save the testing classification report
testing_report = classification_report(y_test, testing_predictions, target_names=target_names)

# Print the testing classification report
print(testing_report)

              precision    recall  f1-score   support

 bad quality       0.91      0.89      0.90       355
good quality       0.25      0.29      0.27        45

    accuracy                           0.82       400
   macro avg       0.58      0.59      0.59       400
weighted avg       0.83      0.82      0.83       400



## Logistic Regression - `solver='liblinear'`

In [ ]:
# instantiate the Logistic Regression model as classifier

classifier = LogisticRegression(solver='liblinear',
                                max_iter=200,
                                random_state=1)
classifier

LogisticRegression(max_iter=200, random_state=1, solver='liblinear')

In [ ]:
# Fit model using scaled training data

lr_model = classifier.fit(X_train_scaled, y_train)
lr_model


LogisticRegression(max_iter=200, random_state=1, solver='liblinear')

## Create predictded values for test and train

In [ ]:
#Generate training predictions
# training_predictions = lr_model.predict(X_train.values)

#Generate testing predictions
testing_predictions = classifier.predict(X_test.values)

## Confusion Matrix for testing data

In [ ]:
# Create and save the confusion matrix for the testing data
test_matrix = confusion_matrix(y_test, testing_predictions)

# Print the confusion matrix for the testing data
print(test_matrix)

[[317  38]
 [ 31  14]]


## Classification report for testing

In [ ]:
# Create and save the testing classification report
testing_report = classification_report(y_test, testing_predictions, target_names=target_names)

# Print the testing classification report
print(testing_report)

              precision    recall  f1-score   support

 bad quality       0.91      0.89      0.90       355
good quality       0.27      0.31      0.29        45

    accuracy                           0.83       400
   macro avg       0.59      0.60      0.60       400
weighted avg       0.84      0.83      0.83       400



## Logistic Regression with `RandomOverSampler` from imblearn

In [141]:
# Import the RandomOverSampler module form imbalanced-learn

# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
ros = RandomOverSampler(random_state=1)
# Fit the original training data to the random_oversampler model
X_res, y_res = ros.fit_resample(X_train, y_train)

In [142]:
# Count the distinct values of the resampled labels data
y_res.value_counts()

0    1027
1    1027
Name: target, dtype: int64

## `LogisticRegression` classifier

In [146]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
logistic_regression_model = LogisticRegression(random_state=1,
                                               max_iter=1000)

# Fit the model using the resampled training data
lr_model_res = logistic_regression_model.fit(X_res, y_res)

# Make a prediction using the testing data
predictions_res = lr_model_res.predict(X_test)

## Confusion Matrix for testing data

In [148]:
# Create and save the confusion matrix for the testing data
test_matrix = confusion_matrix(y_test, predictions_res)

# Print the confusion matrix for the testing data
print(test_matrix)

[[274  81]
 [  7  38]]


## Classification report for testing

In [149]:
print(classification_report(y_test, predictions_res, target_names=target_names))


              precision    recall  f1-score   support

 bad quality       0.98      0.77      0.86       355
good quality       0.32      0.84      0.46        45

    accuracy                           0.78       400
   macro avg       0.65      0.81      0.66       400
weighted avg       0.90      0.78      0.82       400



## try different models other than Logistic Regression

## random forest

In [150]:
rf_classifier = RandomForestClassifier()
rf_model = rf_classifier.fit(X_train, y_train)
rf_predict= rf_classifier.predict(X_test)

In [151]:
rf_conf_matrix = confusion_matrix(y_test, rf_predict)
rf_acc_score = accuracy_score(y_test, rf_predict)
print(rf_conf_matrix)
print(rf_acc_score*100)

[[346   9]
 [ 22  23]]
92.25


In [152]:
#Generate training predictions
training_predictions = rf_model.predict(X_train)

#Generate testing predictions
testing_predictions = rf_classifier.predict(X_test)

## Confusion Matrix for testing data

In [153]:
# Create and save the confusion matrix for the testing data
test_matrix = confusion_matrix(y_test, testing_predictions)

# Print the confusion matrix for the testing data
print(test_matrix)

[[346   9]
 [ 22  23]]


## Classification report for testing

In [154]:
# Create and save the testing classification report
testing_report = classification_report(y_test, testing_predictions, target_names=target_names)

# Print the testing classification report
print(testing_report)

              precision    recall  f1-score   support

 bad quality       0.94      0.97      0.96       355
good quality       0.72      0.51      0.60        45

    accuracy                           0.92       400
   macro avg       0.83      0.74      0.78       400
weighted avg       0.92      0.92      0.92       400



## SVM classifier

In [155]:

lin_svc_classifier = SVC()
lin_svc_model = lin_svc_classifier.fit(X_train, y_train)
lin_svc_predict = lin_svc_classifier.predict(X_test)

In [156]:
lin_svc_conf_matrix = confusion_matrix(y_test, rf_predict)
lin_svc_acc_score = accuracy_score(y_test, rf_predict)
print(lin_svc_conf_matrix)
print(lin_svc_acc_score*100)

[[346   9]
 [ 22  23]]
92.25


In [157]:
#Generate training predictions
training_predictions = lin_svc_model.predict(X_train)

#Generate testing predictions
testing_predictions = lin_svc_classifier.predict(X_test)

## Confusion Matrix for testing data

In [158]:
# Create and save the confusion matrix for the testing data
test_matrix = confusion_matrix(y_test, testing_predictions)

# Print the confusion matrix for the testing data
print(test_matrix)

[[355   0]
 [ 44   1]]


## Classification report for testing

In [159]:
# Create and save the testing classification report
testing_report = classification_report(y_test, testing_predictions, target_names=target_names)

# Print the testing classification report
print(testing_report)

              precision    recall  f1-score   support

 bad quality       0.89      1.00      0.94       355
good quality       1.00      0.02      0.04        45

    accuracy                           0.89       400
   macro avg       0.94      0.51      0.49       400
weighted avg       0.90      0.89      0.84       400

